# Pixelhop

In [43]:
def FLOPs_SSL(input, kernel, stride = 1, maxpooling = False, cw = True, output_dim = -1):
    
    # input [channel ,H, W]
    # kernel [#parentnode, size, size]
    # #parentnode is the selected channel from previous channel
    # #parentnode <= channel
    if cw == False:
        res = 1
        for x in input:

            res *= x
            
        for x in kernel:

            res *= x
        return  res
    
    channel, H, _ = input
    parent_nodes, filter_size, _ = kernel
    # neighborhood construction:
    H = H - filter_size + 1
    # for each channel:
    cwFLOP = H ** 2 * (filter_size ** 2) ** 2 
    FLOPs = cwFLOP * parent_nodes
    print('channelwise, without pooling:', cwFLOP)
    if maxpooling:
        
        if output_dim == -1:
            
            output_dim = filter_size ** 2 * channel
            
        cwFLOP += H ** 2 * output_dim
    print('channelwise, with pooling:', cwFLOP)
    
    return FLOPs

input = [3, 32, 32]
kernel = [3, 3, 3]
stride = 1
pooling = 1
res = FLOPs_SSL(input, kernel, stride, pooling)

print('FLOPs:', res/1e6, 'M')

channelwise, without pooling: 72900
channelwise, with pooling: 97200
FLOPs: 0.2916 M


# GBDT

In [39]:
# model size calculation

depth = 5
num_class = 10
num_tree = 1000

parentnode = 2 ** (depth - 1) * 2 # 2: feature idx for splitting, and splitting point
childnode = 2 ** depth * num_class # prediction result for each class stored in each child node
singletree = parentnode + childnode
modelsize = singletree * num_tree

print('model size:', modelsize/1e6, 'M')

model size: 0.352 M


In [38]:
# FLOPs calculation

depth = 5
num_class = 10
num_tree = 1000
feat_dim = 2000
parent_splitting = feat_dim + 1 # find the feature for splititng and compare with the splitting point
find_childnode = parent_splitting * (depth - 1) # go through depth - 1 parent nodes to find the child node
single_tree = find_childnode + num_class # find the current tree prediction, add to the total prediction
FLOPs = single_tree * num_tree

print('FLOPs:', FLOPs/1e6, 'M')

FLOPs: 8.014 M


# MLP

In [41]:
feat_dim = 500
hidden = 300
num_class = 10

FLOPs = feat_dim * hidden + hidden * num_class
FLOPs *= 2
print('FLOPs:', FLOPs/1e6, 'M')

FLOPs: 0.306 M


# calculate real models:

In [28]:
input = [3, 32, 32]
kernel = [3, 1, 1]
stride = 1
pooling = 0
res = FLOPs_SSL(input, kernel, stride, pooling, cw = 0)

print('FLOPs:', res)

FLOPs: 9216


In [29]:
input = [3, 32, 32]
kernel = [3, 5, 5]
stride = 1
pooling = 1
res = FLOPs_SSL(input, kernel, stride, pooling, output_dim = 42)

print('FLOPs:', res)

FLOPs: 1568784


In [30]:
input = [42, 14, 14]
kernel = [42, 5, 5]
stride = 1
pooling = 1
res = FLOPs_SSL(input, kernel, stride, pooling, output_dim = 254)

print('FLOPs:', res)

FLOPs: 3691800


In [31]:
input = [254, 5, 5]
kernel = [254, 5, 5]
stride = 1
pooling = 0
res = FLOPs_SSL(input, kernel, stride, pooling)

print('FLOPs:', res)

FLOPs: 158750


In [33]:
flops = 9216 + 1568784 + 3691800 + 158750
print('FLOPs:', flops/1e6, 'M')

FLOPs: 5.42855 M


# Calculate HAAR

In [44]:
input = [3, 32, 32]
kernel = [3, 2, 2]
stride = 1
pooling = 1
res = FLOPs_SSL(input, kernel, stride, pooling, cw = 1)

print('FLOPs:', res)

channelwise, without pooling: 15376
channelwise, with pooling: 26908
FLOPs: 80724
